# CL Housing Analysis

# Part 1: Data scraping and preparation

# _*** From project 1 of CMSC641_
### Let's do some scraping

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [44]:
r = requests.get('https://washingtondc.craigslist.org/search/mld/apa?search_distance=10&postal=20740&availabilityMode=0&sale_date=all+dates')
r.text;
soup = BeautifulSoup(r.content, 'lxml');

print(soup.prettify())

In [54]:
link = 'https://washingtondc.craigslist.org/search/mld/apa?search_distance=10&postal=20740&availabilityMode=0&sale_date=all+dates'
def makeSoup(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml');
    return(soup)

- Looking at the source code after searching for all apts/housing for rent within 10 miles of CP, we see that every entry on the MAP starts with <li class="result-row" data-pid="675xxxxxxx".
- 



In [70]:
# LETS GET SOME ATTRIBUTES
# Using a BeautifulSoup object, we extract in our first pass the relevant information on the posting that can be obtained 
# before visiting the link.
def getHouses(soup):
    sum_Unlabeled = 0 
    r_matrix = []
    for row in soup.find_all(class_="result-row"):
        #Try is to avoid Nonetype object error (can't call .text)
        try:
            pID = row.get('data-pid')

            rtitle = row.find(class_="result-title hdrlnk") #Grabs post title + href
            rtitle_txt = rtitle.text
            price = row.find(class_='result-price').text
            href = rtitle.get('href')
            date = row.find(class_="result-date").get('datetime') #print(date) [Format: 2018-12-11 13:57]
            r = row.find(class_='housing').text.splitlines() # for each row, grab BR + SQFT
            #print(r)

            #Removes whitespace/empty lines
            brSqft = [i.replace(" ",'').replace('-','') for i in r if not (i.isspace()) and i != '']
            #If number of bedrooms is N/A, make it nan
            if(len(brSqft)==1):
                brSqft = [float('nan')] + brSqft
            #print(brSqft)

        except: 
            sum_Unlabeled+=1 #Catch value error, some don't have text
        r_matrix.append([pID, rtitle_txt, price] + brSqft + [href])
    #print(r_matrix[:10])
    #df = pd.DataFrame(r_matrix)
    #print(r_matrix[5])
    #print(df.shape)
    #print(df[:,0])
    print("Number of ^bad^ rows: " + str(sum_Unlabeled))
    return(r_matrix)
df = getHouses(makeSoup(link))



Number of ^bad^ rows: 3



# Part 1B:
## Issue
There are only 120 rows that we access can access from the source code when looking at postings through the map. Therefore, we should use a different format where we can access all of the ~11000 postings. Changing to the 'list' mode, we see that there are still only 120 rows, and to advance we'll simply have to view the next page. 

Examining the source code, the next link is:
<link rel="next" href="https://washingtondc.craigslist.org/search/apa?availabilityMode=0&postal=20740&s=120&search_distance=10">

and the following set of postings will be at the link:
<link rel="next" href="https://washingtondc.craigslist.org/search/apa?availabilityMode=0&postal=20740&s=240&search_distance=10">

So, what we will have to do to grab the data is repeat the data scraping procedure above on each webpage, replacing s=120, with s=240, s=360, etc. Until our GET request returns an error

In [72]:
results = []
strkey = ''
#Max results in craigslist always appears to be for 3000 pages
for i in range(0,360,120):#3000,120):
    if(i!=0):
        strkey = 's=' + str(i)
    r = requests.get('https://washingtondc.craigslist.org/search/apa?availabilityMode=0&postal=20740&' + strkey + '&search_distance=10')
    r_matrix = getHouses(makeSoup(link))
    
    results.append(r_matrix)
    #if(i==0):
    #    print(r.text)

print(np.array(results).shape)
print(results)
    #df.append(df_current, ignore_index=True)
        
        

Number of ^bad^ rows: 3
Number of ^bad^ rows: 3
Number of ^bad^ rows: 3
(3, 120, 6)
[[['6770437132', 'Stunning Studio With One Month Free/Park View/Utilities Included!!', '$1599', nan, '450ft2', 'https://washingtondc.craigslist.org/doc/apa/d/stunning-studio-with-one/6770437132.html'], ['6770436493', 'Gorgeous 1 BR 1 BA Apartment', '$2210', '1br', '810ft2', 'https://washingtondc.craigslist.org/mld/apa/d/gorgeous-1-br-1-ba-apartment/6770436493.html'], ['6770436477', 'Open House Saturday, December 15th 10:00 am to 4:00 pm', '$2500', '1br', '600ft2', 'https://washingtondc.craigslist.org/doc/apa/d/open-house-saturday-december/6770436477.html'], ['6770436486', 'Lovely Newly Renovated Studio/Park View/ONE MONTH FREE RENT!!', '$1675', nan, '450ft2', 'https://washingtondc.craigslist.org/doc/apa/d/lovely-newly-renovated-studio/6770436486.html'], ['6770429291', 'Close to Restaurants, Billiards Table, Clubhouse', '$2227', '2br', '1101ft2', 'https://washingtondc.craigslist.org/mld/apa/d/close-to-re

# PART 2: Storing in SQL (super basic)

In [72]:
#Super basic example of how to write to a SQLite DB
from sqlalchemy import create_engine
engine = create_engine('sqlite:///foo.db') #Already created
connection = engine.connect()

print(engine.execute("SELECT * FROM Listings").fetchall())

df.to_sql('Listings', con=engine,if_exists='replace')
connection.close()
#Fetch all Listings results [ID, PID, Title, href, BR, Sqft]
#engine.execute("SELECT * FROM Listings").fetchall()

[(0, '6759804303', 'Hardwood Flooring, Spacious Floorplans, Dog Park', 'https://washingtondc.craigslist.org/mld/apa/d/hardwood-flooring-spacious/6759804303.html', '1br', '780ft2'), (1, '6759009217', 'Corner Three Bedroom Available, Full Size W/D, SS Appl, Kitch Island', 'https://washingtondc.craigslist.org/doc/apa/d/corner-three-bedroom/6759009217.html', '1br', '780ft2'), (2, '6744703743', 'Over-Sized 2 Bedroom 1.5 Bath W/D Inside Amazing Balcony View', 'https://washingtondc.craigslist.org/mld/apa/d/over-sized-2-bedroom-15-bath/6744703743.html', '1br', '780ft2'), (3, '6747400318', '"Bicycle storage and maintenance facilities"', 'https://washingtondc.craigslist.org/mld/apa/d/bicycle-storage-and/6747400318.html', '1br', '780ft2'), (4, '6760088996', '24 Hour Front Desk, Walk to PG Plaza Metro Rail, Enormous Bedrooms', 'https://washingtondc.craigslist.org/mld/apa/d/24-hour-front-desk-walk-to-pg/6760088996.html', '1br', '780ft2'), (5, '6745487927', 'Pool, Cable Ready, Laundry Service', 'htt

In [43]:
#Example ROW HTML CL
<li class="result-row" data-pid="6766765340">
       <a class="result-image gallery" data-ids="1:00T0T_l3wW2i19dU4,1:00808_27z8dUOmSMc,1:00R0R_kZgzYnaiyS1,1:00F0F_6vlKEH1eVly,1:00505_iYmECZZDUSu,1:00B0B_d8JLqKVfaT9,1:00E0E_4gBiNtn6kcx,1:00Y0Y_4LZ3Igh2TzK,1:00g0g_eEcoaDN6LMl,1:00404_xaChvWCrwb,1:00p0p_jLnMTuiLRnR" href="https://washingtondc.craigslist.org/doc/apa/d/beautiful-jr-one-bedroom/6766765340.html">
        <span class="result-price">
         $1900
        </span>
       </a>
       <p class="result-info">
        <span class="icon icon-star" role="button">
         <span class="screen-reader-text">
          favorite this post
         </span>
        </span>
        <time class="result-date" datetime="2018-12-06 23:50" title="Thu 06 Dec 11:50:20 PM">
         Dec  6
        </time>
        <a class="result-title hdrlnk" data-id="6766765340" href="https://washingtondc.craigslist.org/doc/apa/d/beautiful-jr-one-bedroom/6766765340.html">
         A Beautiful Jr. One Bedroom Condo in Sought After Broighton Building
        </a>
        <span class="result-meta">
         <span class="result-price">
          $1900
         </span>
         <span class="housing">
          1br -
                    600ft
          <sup>
           2
          </sup>
          -
         </span>
         <span class="result-hood">
          (Kalorama/ Dupont Circle)
         </span>
         <span class="result-tags">
          pic
     <span class="maptag" data-pid="6766765340">
           map
          </span>
         </span>
         <span class="banish icon icon-trash" role="button">
          <span class="screen-reader-text">
           hide this posting
          </span>
         </span>
         <span aria-hidden="true" class="unbanish icon icon-trash red" role="button">
         </span>
         <a class="restore-link" href="#">
          <span class="restore-narrow-text">
           restore
          </span>
          <span class="restore-wide-text">
           restore this posting
          </span>
         </a>
        </span>
       </p>
      </li>

SyntaxError: invalid syntax (<ipython-input-43-4148719efacc>, line 2)

## Step 2: Tidying
## Here we tidy the data as required. We combine the date and time columns for start, end and max, into single datetime columns through the function makeDateTime was used to parse these, and then the datetime package was used to combine these into datetime objects. We also drop the unneeded columns and set missing values to NaN.

## Step 3: Scrap NASA data
## Here we scrape the data from the nasa site, similarly as before, neglecting the irrelevant lines of text and using pythons string parsing functions to make a 2D list of the solar flare data. We use BeautifulSoup to get the text out of the 'a'  tags. Finally, we import this data into a pandas Dataframe df2, keeping only the important first 14 elements of each line. 

We replace all non-values of the various forms with NaNs and save this to a new dataframe df_nan.

## Step 4: Tidy NASA table
## To tidy this table, we replace all non-values of the various forms with NaNs and save this to a new dataframe df_nan. We then create a column of booleans called isHalo, making the value true if the corresponding row is of a halo flare.  We create another column width_lower_ bound of booleans, filled with True when row's width column contains a lower bound. Finally, we combine date and time columns as we did previously.

# PART 2 Analysis

## Question 1: Replication
#Can you replicate the top 50 solar flare table in SpaceWeatherLive.com #exactly using the data obtained from NASA? 
#That is, if you get the top 50 solar flares from the NASA table based on #their classification (e.g., X28 is the highest), do you get data for the ##same solar flare events?

No, if we just order by classification, we do not exactly replicate the data from SpaceWeatherLive.com using the data from NASA. The first reason is that the NASA data does not include the year 2017, while SpaceWeatherLive.com does. Also, some of the data from NASA has NaN values in the classification section.

[See the code below for the top 50 solar flare table from NASA]


## Question 2: Integration

#Write a function that finds the best matching row in the NASA data for each of the top 50 solar flares in the SpaceWeatherLive.com data. Here, you have to decide for yourself how you determine what is the best matchin entry in the NASA data for each of the top 50 solar flares.

Initial idea:
To find the best matching row in the NASA data for each of the top 50 solar flares on SpaceWeatherLive.com, we will look at matching start dates. Since X-class solar flares only occur about 10 times a year, if the start date of an X-class solar flare matches in the two data sets, then we can assume it is the same solar flare. 

Updated idea:
It turns out that the starting dates don't seem to match up perfectly. After doing some preliminary analysis, I've found that in the top 50 SpaceWeatherLive solar flares, there are no two solar flares of the same classification that also occur during the same year. Therefore, we will look for matches between the two data sets based on two conditions: they have the same classification AND, they occur during the same year. 


## Question 3: Analysis
Plot one plot that shows the top 50 solar flares in context with all data available in the NASA dataset.

I am going to plot the starting frequencies of the NASA dataset over time. 
Flares that appear in the SpaceWeatherLive set (and thus have a rank) will be distinguished by red dots. We will look to see if there is any clear trend as to how solar flare starting frequencies varies with the date & time of occurence. 



The covariance appears to be close to zero here. It looks like the starting frequency of the solar flares is independent of the time at which the solar flares occur. 